In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import os
import random

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
seed = 42
batch_size = 32
input_shape = (256, 256, 3)
epochs = 30

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset_dir = '../input/dataset/training'


labels = ['Apple',        # 0
          'Blueberry',    # 1
          'Cherry',       # 2
          'Corn',         # 3
          'Grape',        # 4
          'Orange',       # 5
          'Peach',        # 6
          'Pepper',       # 7
          'Potato',       # 8
          'Raspberry',    # 9
          'Soybean',      # 10
          'Squash',       # 11
          'Strawberry',   # 12
          'Tomato'        # 13
          ]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        brightness_range=[0.4, 1.5],
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        rescale=1. / 255,
                                        fill_mode='reflect')

train_val_data_gen = ImageDataGenerator(validation_split=0.2, rescale=1. / 255,)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method to create iterator over the indices
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                                        target_size=(256, 256),
                                                        color_mode='rgb',
                                                        classes=None,
                                                        class_mode='categorical',
                                                        batch_size=batch_size,
                                                        shuffle=True,#at the end of the epoch it will shuffle the data
                                                        seed=seed,
                                                        subset='training')

val_gen = train_val_data_gen.flow_from_directory(directory=dataset_dir,
                                                  target_size=(256, 256),
                                                  color_mode='rgb',
                                                  classes=None,
                                                  class_mode='categorical',
                                                  batch_size=batch_size,
                                                  shuffle=True, 
                                                  seed=seed,
                                                  subset='validation')

In [ ]:
# Download and plot the Xception model
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    classes=14, # number of classes to classify images into
    classifier_activation="softmax",
)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(14, activation='softmax')(x)

# Connect input and output through the Model class
model = tfk.Model(inputs=base_model.input, outputs=predictions, name='model')


In [ ]:
callbacks = []

# Tensorboard
tensorboard = TensorBoard(log_dir="tb_logs/")
callbacks.append(tensorboard)
# Early Stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                                restore_best_weights=True)
callbacks.append(es_callback)

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
model.summary()

# Train the model
model.fit(
    x=aug_train_gen,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=val_gen,
    callbacks= callbacks)


model.save("data_augmentation_experiments/CNN_xception_Best")